# Mark multiple boundaries using subdomains

For Dirichlet-BCs, simple functions can define the boundaries:

In [2]:
from fenics import *
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, 'P', 1)

# whole boundary
bc = DirichletBC(V, Constant(1), 'on_boundary')
# only at x = 1:
bc = DirichletBC(V, Constant(1), 'on_boundary && near(x[0], 1)')

# a bit more complex: x = 0 or y = 1
def boundary(x, on_boundary):
    return on_boundary and (near(x[0], 0) or near(x[1], 1))

bc = DirichletBC(V, Constant(1), boundary)

Neumann or Robin BCs, the boundaries have to be marked, by "subclassing" the FEniCS SubDomain class:

In [4]:
# class that defines boundary
class boundary_top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], 1)

# create a MeshFunction that contains boundary information
boundaries = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)
# create object of boundary class and mark the 'boundaries' function with the tag "1"
top = boundary_top()
top.mark(boundaries, 1)

# re-define the Measure ds with the boundary information
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)
# now we can use this in a variational problem:
v = TestFunction(V)
L = Constant(3)*v*ds(1)
# ds(1) integrates only over the boundary marked with "1"
